#CDIA:::AAI - BFS and BFSg

Ibai Laña

**The problem "Wolf, goat, and cabbage"**

https://en.wikipedia.org/wiki/River_crossing_puzzle

A farmer must transport a wolf, a goat and cabbage from one side of a river to another using a boat which can only hold one item in addition to the farmer, subject to the constraints that the wolf cannot be left alone with the goat,  and the goat cannot be left alone with the cabbage

### State definition and comprobations
The state will be defined as a 4-elements (boolean) vector, representing each one of them the position of the elements:

1st position: location of the farmer (LEFT, RIGHT)

2nd position: location of the wolf (LEFT, RIGHT)

3rd position: location of the goat (LEFT, RIGHT)

4th position: location of the cabbage (LEFT, RIGHT)

### Actions, conditions and effects
Now we have an initial state, and a way for checking we are or not in the final one, we can define the way the agent will move in the environment.

4 different actions can be done

1: "farmer" -> changes the side of the farmer (alone)
2: "wolf" -> changes the side of the wolf (and also the farmer)
3: "goat" -> changes the side of the goat (and also the farmer)
4: "cabbage" -> changes the side of the cabbage (and also the farmer)
Considering: moving the farmer can be done if no restrictions are violated. Moving another element can be done if no restrictions are violated and the element is in the same side of the river than the farmer

### Restrictions

For moving the farmer, conditions to be satisfied by the state are wolf and goat are not left alone, as well as goat and cabbage cannot be left togheter. With the goat and the wolf the options are:

| Farmer | Wolf | Goat | OK |
| --- | --- | --- | --- |
| 0|0 |0 |OK |
| 0|0 |1 |OK |
| 0|1 |0 |OK |
| 0|1 |1 |NO OK |
| 1|0 |0 |NO OK |
| 1|0 |1 |OK |
| 1|1|0 |OK |
| 1|1 |1 |OK |

Just the cases where they are together and the farmer is in the contrary state. If they are together but the farmer is there, no probs!

## Problem Formulation.
Parting from the previous steps, we create a data structure to define the problem and a function to solve it. This is the same for any of the solving algorithms

In [ ]:
#we state the problem as a dictionary with the initial state, and the list of possible actions
problem = {"initial_state":{"Farmer":"L","Wolf":"L","Goat":"L","Cabbage":"L"},
           "actions": ["Farmer", "Wolf", "Goat", "Cabbage"]}


## Final state comprobation function


In [ ]:
# if all values of a state ar R, then it is solved
def is_final_state (state):
  return all(value == "R" for value in state.values())

## Restriction comprobation function

In [ ]:
def check_possible_state(state):
  comprobation_wolf_goat = not ((state["Wolf"]==state["Goat"]) & (state["Wolf"]!=state["Farmer"]))
  comprobation_goat_cabbage = not ((state["Cabbage"]==state["Goat"]) & (state["Goat"]!=state["Farmer"]))
  return comprobation_wolf_goat and comprobation_goat_cabbage

## Action function: Move one element to the other side
Movement changes state but there are 2 main restrictions:
1. only farmer can move alone
2. if any of the other elements are moved, they move with farmer(need to be in the same side) and the new state is checked

If the movement is possible, new state is returned. If not,the same state as previous.


In [ ]:
def change_element_status (element, state):
  if state[element]=="R":
        return "L"
  else:
        return "R"

def move (element, state):
  #check if more than 2 agents are moving:
  new_state=state.copy()
  if element=="Farmer":
    #change the status to the new one in the new state
    new_state["Farmer"]=change_element_status(element, state)
    #check if this state is possible:
    if check_possible_state(new_state):
      return new_state
    else:
        print ("Can't move only farmer")
        return state #same as previous
  else: #if any other element is to be moved, first, check if it is in teh same side as the farmer
    if state[element]!=state["Farmer"]:
      print ("can't move "+element+" without farmer")
      return state #same as previous
    else:
      new_state["Farmer"]=change_element_status("Farmer", state)
      new_state[element]=change_element_status(element, state)
      #now, check if this new state is possible:
      if check_possible_state(new_state):
        return new_state
      else:
        print ("Can't move ", element)
        return state #same as previous


## Frontier operation Functions
As we have seen, the frontier is encoded as a FIFO queue, that can be represented in Python as a simple list from where we can add or remove elements.

We define the frontier and the PUSH and PULL methods


In [ ]:
def push_to_frontier (state, frontier):
  frontier.append(state)
  return frontier

def pull_from_frontier (frontier): #gets the first element, removes it from the frontier afterwards
  state = frontier[0]
  frontier = frontier[1:]
  return state, frontier


## Expansion Function
Expanding nodes in this case is trying to move all agents from a particular state. Some movements are not possible, but this is already solved in the movement action script.





In [ ]:
def expand (state, frontier, available_actions):
  # check for every possible action which ones are possible, and add these
  # states to the frontier
  for element in available_actions:
    new_state = move (element, state)
    if new_state!=state:
      #if they are the same, the movement is forbidden.
      #If they are different, the new state is moved to the frontier
      frontier =  push_to_frontier(new_state, frontier)
  return frontier


## BFS formulation

```
1. Make a node with the initial problem state
2. Insert node into the frontier data structure
3. WHILE final state not found AND frontier is not empty DO
  3.1 Remove first node from the frontier
  3.2 IF node contains final state THEN final state found
  3.3 IF node doesn’t contain final state THEN
     3.3.1 EXPAND node’s state
     3.3.2 Insert successor nodes into frontier
4. IF final state found THEN
  4.1  RETURN sequence of actions found
5. ELSE  “solution not found”
```




In [ ]:
def BFS(problem):

  # 1. problem definition
  initial_state=problem["initial_state"]
  available_actions = problem["actions"]
  frontier = []

  sequence_of_states = [] #we create this list to store all the states that the algorithm follows
  state = {} # define state outside while context

  # 2. add node to frontier
  frontier =  push_to_frontier(initial_state, frontier)

  # 3. start exploring and expanding the frontier
  i=1
  while len (frontier)>0: #if we have elements in the frontier...

    # 3.1. get first element of frontier and delete it
    state, frontier = pull_from_frontier (frontier)
    # **** add it to our sequence, to know which elements we have visited and check how is it going
    sequence_of_states.append(state)
    print ("---------------------------------------------------------------")
    print ("iteration ", i)
    print ("states in frontier prior to state exploration", len (frontier))
    print (state)

    # 3.2 check if it is final state:
    if is_final_state (state):
      break #we end while. state will remain this last state computed, and sequence of actions will have all states.
    # 3.3 if it is not final, expand.
    else:
      # 3.3.1, 3.3.2__ Our method expand adds all available successors to frontier
      frontier = expand (state, frontier, available_actions)
    i+=1
  #loop keeps running until no more nodes available or final state obtained

  # 4. if state is final, the while has broken and we get this solution and print the sequence
  if is_final_state (state):
    return (state)
  # 5. else
  else:
    print ("no solution found")





Call the method

In [ ]:
solution = BFS(problem)

---------------------------------------------------------------
iteration  1
states in frontier prior to state exploration 0
{'Farmer': 'L', 'Wolf': 'L', 'Goat': 'L', 'Cabbage': 'L'}
Can't move only farmer
Can't move  Wolf
Can't move  Cabbage
---------------------------------------------------------------
iteration  2
states in frontier prior to state exploration 0
{'Farmer': 'R', 'Wolf': 'L', 'Goat': 'R', 'Cabbage': 'L'}
can't move Wolf without farmer
can't move Cabbage without farmer
---------------------------------------------------------------
iteration  3
states in frontier prior to state exploration 1
{'Farmer': 'L', 'Wolf': 'L', 'Goat': 'R', 'Cabbage': 'L'}
can't move Goat without farmer
---------------------------------------------------------------
iteration  4
states in frontier prior to state exploration 3
{'Farmer': 'L', 'Wolf': 'L', 'Goat': 'L', 'Cabbage': 'L'}
Can't move only farmer
Can't move  Wolf
Can't move  Cabbage
----------------------------------------------------

# GRAPHS
The solution with graphs is very similar, but we need to avoid using already expanded nodes. Thus, the solution goes by maintaining an "expanded" node list and checking each iteration if the node to expand has already been expanded before.

We will change the expansion function to check that list and the BFS function to maintain that list. The rest of methods are the same:


In [ ]:
#problem formulation
problem = {"initial_state":{"Farmer":"L","Wolf":"L","Goat":"L","Cabbage":"L"},
           "actions": ["Farmer", "Wolf", "Goat", "Cabbage"]}

# final state check
def is_final_state (state):
  return all(value == "R" for value in state.values())

#check restrictions
def check_possible_state(state):
  comprobation_wolf_goat = not ((state["Wolf"]==state["Goat"]) & (state["Wolf"]!=state["Farmer"]))
  comprobation_goat_cabbage = not ((state["Cabbage"]==state["Goat"]) & (state["Goat"]!=state["Farmer"]))
  return comprobation_wolf_goat and comprobation_goat_cabbage

# change status
def change_element_status (element, state):
  if state[element]=="R":
        return "L"
  else:
        return "R"
# action
def move (element, state):
  #check if more than 2 agents are moving:
  new_state=state.copy()
  if element=="Farmer":
    #change the status to the new one in the new state
    new_state["Farmer"]=change_element_status(element, state)
    #check if this state is possible:
    if check_possible_state(new_state):
      return new_state
    else:
        print ("Can't move only farmer")
        return state #same as previous
  else: #if any other element is to be moved, first, check if it is in teh same side as the farmer
    if state[element]!=state["Farmer"]:
      print ("can't move "+element+" without farmer")
      return state #same as previous
    else:
      new_state["Farmer"]=change_element_status("Farmer", state)
      new_state[element]=change_element_status(element, state)
      #now, check if this new state is possible:
      if check_possible_state(new_state):
        return new_state
      else:
        print ("Can't move ", element)
        return state #same as previous

# frontier ops--> these are BSF specific, they will be the same for all BFS problems, graph or no graph
def push_to_frontier (state, frontier):
  frontier.append(state)
  return frontier

def pull_from_frontier (frontier): #gets the first element, removes it from the frontier afterwards
  state = frontier[0]
  frontier = frontier[1:]
  return state, frontier




---


Changed functions start here

In [ ]:
def expand (state, frontier, available_actions, expanded_nodes): # CHANGE in the definition of the function to include expanded nodes
  # check for every possible action which ones are possible, and add these
  # states to the frontier
  for element in available_actions:
    new_state = move (element, state)
    if new_state!=state and new_state not in expanded_nodes:### WE ADD THIS CHECK: if it has not been expanded, add to frontier, else don't add.
      #if they are the same, the movement is forbidden.
      #If they are different, the new state is moved to the frontier
      frontier =  push_to_frontier(new_state, frontier)
  return frontier

def BFS_g(problem):

  # 1. problem definition
  initial_state=problem["initial_state"]
  available_actions = problem["actions"]
  frontier = []

  expanded_nodes = [] ## WE CREATE AN EXPANDED NODES LIST

  sequence_of_states = [] #we create this list to store all the states that the algorithm follows
  state = {} # define state outside while context

  # 2. add node to frontier
  frontier =  push_to_frontier(initial_state, frontier)

  # 3. start exploring and expanding the frontier
  i=1
  while len (frontier)>0: #if we have elements in the frontier...

    # 3.1. get first element of frontier and delete it
    state, frontier = pull_from_frontier (frontier)

    #EVERY TIME A NODE IS PULLED FROM THE FRONTIER, IT HAS BEEN EXPANDED. ADD IT TO THE EXPANDED NODES LIST
    expanded_nodes.append(state)

    # **** add it to our sequence, to know which elements we have visited and check how is it going
    sequence_of_states.append(state)
    print ("---------------------------------------------------------------")
    print ("iteration ", i)
    print ("states in frontier prior to state exploration", len (frontier))
    print (state)

    # 3.2 check if it is final state:
    if is_final_state (state):
      break #we end while. state will remain this last state computed, and sequence of actions will have all states.
    # 3.3 if it is not final, expand.
    else:
      # 3.3.1, 3.3.2__ Our method expand adds all available successors to frontier
      frontier = expand (state, frontier, available_actions, expanded_nodes)
    i+=1
  #loop keeps running until no more nodes available or final state obtained

  # 4. if state is final, the while has broken and we get this solution and print the sequence
  if is_final_state (state):
    return (state)
    print (len (expanded_nodes))
  # 5. else
  else:
    print ("no solution found")


In [ ]:
solution = BFS_g(problem)

---------------------------------------------------------------
iteration  1
states in frontier prior to state exploration 0
{'Farmer': 'L', 'Wolf': 'L', 'Goat': 'L', 'Cabbage': 'L'}
Can't move only farmer
Can't move  Wolf
Can't move  Cabbage
---------------------------------------------------------------
iteration  2
states in frontier prior to state exploration 0
{'Farmer': 'R', 'Wolf': 'L', 'Goat': 'R', 'Cabbage': 'L'}
can't move Wolf without farmer
can't move Cabbage without farmer
---------------------------------------------------------------
iteration  3
states in frontier prior to state exploration 0
{'Farmer': 'L', 'Wolf': 'L', 'Goat': 'R', 'Cabbage': 'L'}
can't move Goat without farmer
---------------------------------------------------------------
iteration  4
states in frontier prior to state exploration 1
{'Farmer': 'R', 'Wolf': 'R', 'Goat': 'R', 'Cabbage': 'L'}
Can't move only farmer
can't move Cabbage without farmer
-------------------------------------------------------